In [1]:
import pickle
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

from utils import seed_everything, set_device, get_train_val_dataloaders, train
from model import FeedForwardRegressionNet

from torch.utils.data import TensorDataset
from torch.optim import Adam

In [2]:
n_jobs = 1000
results = []

for i in range(n_jobs):
    results.append(pickle.load(open(f"results/training_stats_{i}", "rb")))

In [3]:
df = pd.DataFrame(results).sort_values("Valid. Loss")

In [4]:
df[0:10]

,Training Loss,Valid. Loss,Training Time,Validation Time,hidden_dims,learning_rate,embed_dim0,embed_dim1,epochs,job_id
435,5867.846587,6208.250257,0:00:02,0:00:00,"[128, 16, 8, 64]",0.003,64,32,32,435
979,7193.813212,6343.188381,0:00:01,0:00:00,"[16, 128]",0.010,64,32,32,979
646,6305.614021,6951.190204,0:00:01,0:00:00,"[64, 128]",0.010,4,8,32,646
586,7612.546547,6982.667661,0:00:02,0:00:00,"[128, 16, 4, 64]",0.003,64,16,32,586
855,6707.967722,7035.854330,0:00:03,0:00:00,"[128, 8, 16]",0.003,32,4,32,855
379,7644.281610,7630.083813,0:00:02,0:00:00,"[128, 16, 8, 64]",0.001,8,32,32,379
2,9952.709217,7930.797397,0:00:02,0:00:00,"[128, 8, 16]",0.010,8,16,16,2
926,7908.773299,7972.807845,0:00:02,0:00:00,"[64, 128, 4, 16]",0.001,4,16,32,926
106,8025.069873,8026.301784,0:00:03,0:00:00,"[16, 64, 128]",0.010,8,64,32,106
607,9456.042930,8040.701280,0:00:02,0:00:00,"[8, 128, 4, 64]",0.003,64,8,32,607


In [5]:
best_hyperparameters = {
    "hidden_dims": [64, 128],
    "learning_rate": 0.01,
    "embed_dim0": 4,
    "embed_dim1": 8,
    "epochs": 32,
}

pickle.dump(best_hyperparameters, open("best_hyperparameters.pkl", "wb"))

In [6]:
hyperparameters = pickle.load(open("best_hyperparameters.pkl", "rb"))

HIDDEN_DIMS = hyperparameters["hidden_dims"]
LEARNING_RATE = hyperparameters["learning_rate"]
EMBED_DIM0 = hyperparameters["embed_dim0"]
EMBED_DIM1 = hyperparameters["embed_dim1"]
EPOCHS = hyperparameters["epochs"]

batch_size = 64
val_size = 0.1

In [7]:
device = set_device()

X = pickle.load(open("preprocessed_data/X_train_tensor.pkl", "rb"))
y = pickle.load(open("preprocessed_data/y_train_tensor.pkl", "rb"))

dataset = TensorDataset(X, y)

train_dataloader, validation_dataloader = get_train_val_dataloaders(
    dataset, val_size, batch_size
)

No GPU available, using the CPU instead.
45,000 training samples
4,999 validation samples


In [8]:
input_dim = X.shape[1] - 2
vocab_size0 = len(np.unique(X[:, 0]))
vocab_size1 = len(np.unique(X[:, 1]))
output_dim = 1

model = FeedForwardRegressionNet(
    input_dim,
    vocab_size0,
    vocab_size1,
    EMBED_DIM0,
    EMBED_DIM1,
    HIDDEN_DIMS,
    output_dim,
)

In [9]:
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

criterion = nn.MSELoss()

seed_everything(42)

training_stats = train(
    model,
    EPOCHS,
    train_dataloader,
    validation_dataloader,
    criterion,
    optimizer,
    device=device,
    SSY=y.var() * len(y),
)

torch.save(model.state_dict, "best_model.pt")


======== Epoch 1 / 32 ========
Training...
  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 247003.97
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 33885.38
  Validation SSE: 170614010.38
  Valida

  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 17024.80
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 17807.73
  Validation SSE: 89872833.46
  Validation took: 0:00:00
  Validation R^2: 0.9961900115013123

======== Epoch 10 / 32 ========
Trai

  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 10356.98
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 10467.07
  Validation SSE: 52795208.12
  Validation took: 0:00:00
  Validation R^2: 0.9977618

  Batch    40  of    704.    Elapsed: 0:00:00.
  Batch    80  of    704.    Elapsed: 0:00:00.
  Batch   120  of    704.    Elapsed: 0:00:00.
  Batch   160  of    704.    Elapsed: 0:00:00.
  Batch   200  of    704.    Elapsed: 0:00:00.
  Batch   240  of    704.    Elapsed: 0:00:00.
  Batch   280  of    704.    Elapsed: 0:00:01.
  Batch   320  of    704.    Elapsed: 0:00:01.
  Batch   360  of    704.    Elapsed: 0:00:01.
  Batch   400  of    704.    Elapsed: 0:00:01.
  Batch   440  of    704.    Elapsed: 0:00:01.
  Batch   480  of    704.    Elapsed: 0:00:01.
  Batch   520  of    704.    Elapsed: 0:00:01.
  Batch   560  of    704.    Elapsed: 0:00:01.
  Batch   600  of    704.    Elapsed: 0:00:01.
  Batch   640  of    704.    Elapsed: 0:00:01.
  Batch   680  of    704.    Elapsed: 0:00:01.

  Average training loss: 7339.82
  Training epoch took: 0:00:01

Running Validation...
  Validation Loss: 6480.07
  Validation SSE: 32731064.11
  Validation took: 0:00:00
  Validation R^2: 0.998612403